This notebook is used to postprocess and compare the LE simulations of different modeling configurations

In [4]:
# TODO: A more generic way is needed here to load the path
import sys

sys.path.append(
    "/Users/jian449/Library/CloudStorage/OneDrive-PNNL/Codes/jax-watershed/src"
    # "/global/homes/p/peishi89/jax-watershed/src"
)

In [20]:
import pickle
import pandas as pd
from jax_canoak.subjects import get_met_forcings, get_obs
from jax_canoak.shared_utilities import compute_metrics

In [6]:
site = "US-Hn1"

# Calibration period
f_forcing = f"../../data/fluxtower/{site}/{site}-forcings.csv"
f_obs = f"../../data/fluxtower/{site}/{site}-fluxes-nonegle.csv"

# Test period
f_forcing_test = f"../../data/fluxtower/{site}/{site}-forcings-test.csv"
f_obs_test = f"../../data/fluxtower/{site}/{site}-fluxes-nonegle-test.csv"

# Load data

## Load forcings and observations

In [7]:
# Train
obs = get_obs(f_obs)
met, n_time = get_met_forcings(f_forcing)

# Test
obs_test = get_obs(f_obs_test)
met_test, n_time = get_met_forcings(f_forcing_test)

/Users/jian449/Library/CloudStorage/OneDrive-PNNL/Codes/jax-watershed/src/jax_canoak/subjects/initialization_update.py:270: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  obs.interpolate(method="linear", limit_direction="both", inplace=True)
/Users/jian449/Library/CloudStorage/OneDrive-PNNL/Codes/jax-watershed/src/jax_canoak/subjects/initialization_update.py:270: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  obs.interpolate(method="linear", limit_direction="both", inplace=True)


## Load simulations

In [8]:
# No DL
key1 = "nodl"
f_out1 = f"{site}_{key1}.pkl"
out1 = pickle.load(open(f_out1, "rb"))

In [9]:
# Pure DL
key2 = "dl"
f_out2 = f"{site}_{key2}.pkl"
out2 = pickle.load(open(f_out2, "rb"))

In [10]:
# Hybrid
key3 = "hybridRH"
f_out3 = f"{site}_{key3}.pkl"
out3 = pickle.load(open(f_out3, "rb"))

## Combine 

In [16]:
out2["train"]["LE"].shape

(18960, 1)

In [17]:
le_train = pd.DataFrame(
    data={
        "obs": obs.LE,
        key1: out1["train"]["can.LE"],
        key2: out2["train"]["LE"],
        key3: out3["train"]["can.LE"],
    },
    index=out1["train"]["time"],
)
le_test = pd.DataFrame(
    data={
        "obs": obs_test.LE,
        key1: out1["test"]["can.LE"],
        key2: out2["test"]["LE"],
        key3: out3["test"]["can.LE"],
    },
    index=out1["test"]["time"],
)

# Compute the performances

In [34]:
metrics_keys = ["rmse", "mse", "r2", "kge", "mkge", "cc", "nse"]
metrics_df = pd.DataFrame(columns=metrics_keys, index=[key1, key2, key3])
metrics_test_df = pd.DataFrame(columns=metrics_keys, index=[key1, key2, key3])
for key in [key1, key2, key3]:
    metrics = compute_metrics(le_train["obs"], le_train[key])
    metrics_test = compute_metrics(le_test["obs"], le_test[key])
    for m in metrics_keys:
        metrics_df.loc[key, m] = metrics[m]
    for m in metrics_keys:
        metrics_test_df.loc[key, m] = metrics_test[m]

# Plot